In [58]:
import rioxarray as rio
import numpy as np
import pandas as pd
from tqdm import tqdm
from shapely.geometry import box
import pygc
import geopandas as gpd

In [72]:
da = rio.open_rasterio("./ccidataset(2020)/S40E170_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2020-fv4.0.tiff")
da

<xarray.DataArray (band: 1, y: 11250, x: 11250)>
[126562500 values with dtype=uint16]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 170.0 170.0 170.0 170.0 ... 180.0 180.0 180.0 180.0
  * y            (y) float64 -40.0 -40.0 -40.0 -40.0 ... -50.0 -50.0 -50.0 -50.0
    spatial_ref  int32 0
Attributes: (12/46)
    AREA_OR_POINT:              Area
    cdm_data_type:              INT
    comment:                    These data were produced at ESA CCI as part o...
    Conventions:                CF-1.7
    creator_email:              santoro@gamma-rs.ch
    creator_name:               GAMMA Remote Sensing
    ...                         ...
    time_coverage_resolution:   P1Y
    time_coverage_start:        20200101T000000Z
    title:                      ESA CCI above-ground biomass product level 4,...
    tracking_id:                cd5cb1d2-e582-41ab-a0fc-637f050baf9e
    scale_factor:               1.0
    add_offset:                 0.0

In [73]:
#lengths in m
ew_width = 2000
ns_height = 2000
size = int(ew_width/1000)

main_datafile_path = "../../estingAustralia.csv"

In [75]:
treecoords = pd.read_csv(main_datafile_path)

In [76]:
batch = treecoords.loc[(treecoords["lat"] <= -40) &(treecoords["lat"] > -50) & (treecoords["long"] >= 170) & (treecoords["long"] < 180)]
batch

,Unnamed: 0,project,site,lat,long,Granule Number


In [64]:
# sitelist = treecoords['site']
# lat = list(treecoords['lat'])
# lon = list(treecoords['long'])
# projectlist = list(treecoords["project"])

sitelist = batch['site']
lat = list(batch['lat'])
lon = list(batch['long'])
projectlist = list(batch["project"])

In [65]:
def latLonBoxByWandH(lat,lon,ew_width,ns_height):
    lats, lons = [], []
    #distance in m, az (in deg), lat (in deg), long (in deg)

    res = pygc.great_circle(distance=ew_width/2, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']

    res = pygc.great_circle(distance=ns_height/2, azimuth=180, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=270, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ns_height, azimuth=0, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)
    
    return {'lats':lats,'lons':lons}

In [66]:
#loc_lon,loc_lat
loc_lat = np.asfarray(lat,float)
loc_lon = np.asfarray(lon,float)
len(loc_lat),len(loc_lon)

(95, 95)

In [67]:
for lat, lon, code in tqdm(zip(loc_lat,loc_lon, sitelist), total = len(loc_lat)):
    boxe = latLonBoxByWandH(lat,lon,ew_width,ns_height)
    # print(boxe)
    
    geof = gpd.GeoDataFrame(
        geometry=[
            box(min(boxe["lons"]), 
                min(boxe["lats"]), 
                max(boxe["lons"]), 
                max(boxe["lats"])
                )
            ], 
        crs="EPSG:4326")
    # print(geof.geometry)
    clipped = da.rio.clip(geof.geometry.values, geof.crs, drop=True)
    # clipped.plot()
    
    clipped.rio.to_raster(f"outputCCI/{code}_2017_agbm.tif", dtype="float32")
    # clipped.rio.to_raster("clipped_invert.tif", compress='LZMA', tiled=True, dtype="int32")

100%|██████████| 95/95 [01:16<00:00,  1.24it/s]
